# Feature Engineering for Financial ML Pipeline

This notebook creates comprehensive features for predicting:
- Next best action for client engagement
- Client conversion likelihood
- Client churn risk

## Features Created:
1. **Engagement Metrics** - Web visits, email interactions, campaign responses
2. **Financial Indicators** - Income, assets, retirement readiness
3. **Behavioral Patterns** - Preferences, lifecycle stage, engagement trends
4. **Target Variables** - Conversion, churn, next best action

## Workflow:
1. Run Cells 1-10 to create all features
2. Run Cell 11 to register in Feature Store (if needed for UI visibility)

In [ ]:
# Import required libraries
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.window import Window
from snowflake.ml.feature_store import FeatureStore, Entity, FeatureView
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Get active session
session = snowpark.session._get_active_session()

print(f"🔧 Snowflake Feature Engineering Pipeline")
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")
print(f"Warehouse: {session.get_current_warehouse()}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Verify data availability and ensure correct schema
print(f"\n🔍 Checking data availability...")

# Check current schema and find data tables
current_schema = session.get_current_schema()
print(f"📍 Current schema: {current_schema}")

try:
    client_count = session.sql("SELECT COUNT(*) as count FROM clients").collect()[0]['COUNT']
    event_count = session.sql("SELECT COUNT(*) as count FROM marketing_events").collect()[0]['COUNT']
    
    print(f"\n✅ Data Available in {current_schema}:")
    print(f"📊 Clients: {client_count:,}")
    print(f"📊 Marketing Events: {event_count:,}")
    
    # Check marketing_events table structure
    print("\n🔍 Marketing Events Table Structure:")
    session.sql("DESCRIBE TABLE marketing_events").show()
    
except Exception as e:
    print(f"⚠️ Data tables not found in {current_schema}: {e}")
    print("🔄 Attempting to find data in ML_PIPELINE schema...")
    
    try:
        # Try ML_PIPELINE schema
        session.sql("USE SCHEMA ML_PIPELINE").collect()
        client_count = session.sql("SELECT COUNT(*) as count FROM clients").collect()[0]['COUNT']
        event_count = session.sql("SELECT COUNT(*) as count FROM marketing_events").collect()[0]['COUNT']
        
        print(f"\n✅ Data Found in ML_PIPELINE schema:")
        print(f"📊 Clients: {client_count:,}")
        print(f"📊 Marketing Events: {event_count:,}")
        
        print("\n🔍 Marketing Events Table Structure:")
        session.sql("DESCRIBE TABLE marketing_events").show()
        
    except Exception as e2:
        print(f"❌ Data tables not found in any schema: {e2}")
        print("📋 Please run the data generation notebook (01_Data_Generation_Snowflake.ipynb) first")
        print("   This will create the required CLIENTS and MARKETING_EVENTS tables")


## Step 1: Create Engagement Features


In [ ]:
# Create comprehensive engagement features using Snowflake SQL
print("🎯 Creating engagement features across multiple time windows...")

engagement_features_sql = """
CREATE OR REPLACE TABLE engagement_features AS
WITH time_windows AS (
  SELECT 
    client_id,
    
    -- 7-day engagement metrics
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -7, CURRENT_TIMESTAMP()) THEN 1 END) as total_events_7d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -7, CURRENT_TIMESTAMP()) AND event_type = 'web_visit' THEN 1 END) as web_visits_7d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -7, CURRENT_TIMESTAMP()) AND event_type = 'email_open' THEN 1 END) as email_opens_7d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -7, CURRENT_TIMESTAMP()) AND event_type = 'email_click' THEN 1 END) as email_clicks_7d,
    
    -- 30-day engagement metrics
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) THEN 1 END) as total_events_30d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND event_type = 'web_visit' THEN 1 END) as web_visits_30d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND event_type = 'email_open' THEN 1 END) as email_opens_30d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND event_type = 'email_click' THEN 1 END) as email_clicks_30d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND event_type = 'advisor_meeting' THEN 1 END) as personal_interactions_30d,
    
    -- 90-day engagement metrics
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -90, CURRENT_TIMESTAMP()) THEN 1 END) as total_events_90d,
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -90, CURRENT_TIMESTAMP()) AND event_type = 'web_visit' THEN 1 END) as web_visits_90d,
    
    -- Session quality metrics
    AVG(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND time_on_page IS NOT NULL 
             THEN time_on_page END) as avg_session_duration_30d,
    MAX(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND time_on_page IS NOT NULL 
            THEN time_on_page END) as max_session_duration_30d,
    
    -- Engagement consistency
    COUNT(DISTINCT CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) 
                        THEN DATE(event_timestamp) END) as active_days_30d,
    
    -- Touchpoint value (if column exists, otherwise use default)
    SUM(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) 
             THEN COALESCE(touchpoint_value, 0.5) END) as total_touchpoint_value_30d,
    AVG(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) 
             THEN COALESCE(touchpoint_value, 0.5) END) as avg_touchpoint_value_30d,
    
    -- Conversion indicators (if column exists)
    COUNT(CASE WHEN event_timestamp >= DATEADD(day, -30, CURRENT_TIMESTAMP()) AND COALESCE(conversion_flag, FALSE) = TRUE 
               THEN 1 END) as conversions_30d,
    
    -- Activity recency
    MAX(event_timestamp) as last_activity_timestamp,
    DATEDIFF(day, MAX(event_timestamp), CURRENT_TIMESTAMP()) as days_since_last_activity
    
  FROM marketing_events 
  GROUP BY client_id
),

calculated_metrics AS (
  SELECT 
    *,
    -- Engagement frequency calculations
    CASE WHEN active_days_30d > 0 THEN total_events_30d::DECIMAL / active_days_30d ELSE 0 END as engagement_frequency_30d,
    
    -- Email engagement rates
    CASE WHEN email_opens_30d > 0 THEN email_clicks_30d::DECIMAL / email_opens_30d ELSE 0 END as email_click_rate_30d,
    
    -- Trend indicators (comparing recent vs older activity)
    CASE WHEN total_events_90d > 0 THEN total_events_30d::DECIMAL / (total_events_90d / 3) ELSE 0 END as engagement_trend_30d,
    
    -- Engagement score (composite metric)
    LEAST(1.0, 
      (total_events_30d * 0.3 + 
       web_visits_30d * 0.2 + 
       email_opens_30d * 0.2 + 
       personal_interactions_30d * 0.3) / 100
    ) as engagement_score_30d
    
  FROM time_windows
)

SELECT 
  client_id,
  CURRENT_TIMESTAMP() as feature_timestamp,
  
  -- Raw engagement counts
  total_events_7d, web_visits_7d, email_opens_7d, email_clicks_7d,
  total_events_30d, web_visits_30d, email_opens_30d, email_clicks_30d, personal_interactions_30d,
  total_events_90d, web_visits_90d,
  
  -- Quality metrics
  ROUND(avg_session_duration_30d, 2) as avg_session_duration_30d,
  max_session_duration_30d,
  active_days_30d,
  
  -- Value metrics
  ROUND(total_touchpoint_value_30d, 4) as total_touchpoint_value_30d,
  ROUND(avg_touchpoint_value_30d, 4) as avg_touchpoint_value_30d,
  conversions_30d,
  
  -- Recency
  last_activity_timestamp,
  days_since_last_activity,
  
  -- Calculated metrics
  ROUND(engagement_frequency_30d, 4) as engagement_frequency_30d,
  ROUND(email_click_rate_30d, 4) as email_click_rate_30d,
  ROUND(engagement_trend_30d, 4) as engagement_trend_30d,
  ROUND(engagement_score_30d, 4) as engagement_score_30d
  
FROM calculated_metrics
"""

# Execute feature creation
session.sql(engagement_features_sql).collect()

# Verify results
engagement_count = session.sql("SELECT COUNT(*) as count FROM engagement_features").collect()[0]['COUNT']
print(f"✅ Created engagement features for {engagement_count:,} clients")

# Show sample features
print("\n📊 Sample engagement features:")
session.sql("""
    SELECT client_id, total_events_30d, web_visits_30d, email_opens_30d, 
           engagement_frequency_30d, engagement_score_30d, days_since_last_activity
    FROM engagement_features 
    WHERE total_events_30d > 0
    ORDER BY engagement_score_30d DESC 
    LIMIT 10
""").show()


## Step 2: Create Financial & Behavioral Features


In [ ]:
# Create financial profile and behavioral features
print("💰 Creating financial and behavioral features...")

financial_behavioral_sql = """
CREATE OR REPLACE TABLE financial_behavioral_features AS
WITH client_behaviors AS (
  SELECT 
    me.client_id,
    
    -- Channel preferences
    COUNT(CASE WHEN me.channel = 'Website' THEN 1 END) as web_preference_count,
    COUNT(CASE WHEN me.channel = 'Email' THEN 1 END) as email_preference_count,
    COUNT(CASE WHEN me.channel = 'Phone' THEN 1 END) as phone_preference_count,
    COUNT(CASE WHEN me.channel = 'In-Person' THEN 1 END) as inperson_preference_count,
    
    -- Device preferences
    COUNT(CASE WHEN me.device_type = 'Desktop' THEN 1 END) as desktop_usage,
    COUNT(CASE WHEN me.device_type = 'Mobile' THEN 1 END) as mobile_usage,
    COUNT(CASE WHEN me.device_type = 'Tablet' THEN 1 END) as tablet_usage,
    
    -- Behavioral patterns
    COUNT(*) as total_lifetime_events,
    COUNT(CASE WHEN me.event_type = 'document_download' THEN 1 END) as education_engagement,
    COUNT(CASE WHEN me.event_type = 'advisor_meeting' THEN 1 END) as advisor_meetings_total,
    AVG(me.touchpoint_value) as avg_touchpoint_value,
    
    -- Engagement span
    DATEDIFF(day, MIN(me.event_timestamp), MAX(me.event_timestamp)) as engagement_span_days
    
  FROM marketing_events me
  GROUP BY me.client_id
),

financial_profile AS (
  SELECT 
    c.client_id,
    c.age,
    c.annual_income,
    c.current_401k_balance,
    c.years_to_retirement,
    c.total_assets_under_management,
    c.client_tenure_months,
    c.service_tier,
    c.risk_tolerance,
    c.investment_experience,
    
    -- Financial ratios and scores
    ROUND(c.annual_income::DECIMAL / GREATEST(c.age, 25), 2) as income_to_age_ratio,
    ROUND(c.total_assets_under_management::DECIMAL / GREATEST(c.annual_income, 1), 4) as assets_to_income_ratio,
    
    -- Retirement readiness (simplified model)
    LEAST(1.0, GREATEST(0.0, 
      c.current_401k_balance::DECIMAL / GREATEST((c.annual_income * 10), 1)
    )) as retirement_readiness_score,
    
    -- Wealth growth potential
    LEAST(1.0, 
      ((65 - c.age) / 40 * 0.3) + 
      (LN(c.annual_income) / LN(200000) * 0.4) + 
      (LN(GREATEST(c.total_assets_under_management, 1)) / LN(1000000) * 0.3)
    ) as wealth_growth_potential,
    
    -- Premium client indicator
    CASE WHEN c.total_assets_under_management > 100000 THEN 1 ELSE 0 END as premium_client_indicator,
    
    -- Service tier numeric
    CASE c.service_tier 
      WHEN 'Basic' THEN 1 
      WHEN 'Premium' THEN 2 
      WHEN 'Elite' THEN 3 
      ELSE 0 
    END as service_tier_numeric,
    
    -- Risk tolerance numeric
    CASE c.risk_tolerance 
      WHEN 'Conservative' THEN 1 
      WHEN 'Moderate' THEN 2 
      WHEN 'Aggressive' THEN 3 
      ELSE 0 
    END as risk_tolerance_numeric,
    
    -- Investment experience numeric
    CASE c.investment_experience 
      WHEN 'Beginner' THEN 1 
      WHEN 'Intermediate' THEN 2 
      WHEN 'Advanced' THEN 3 
      ELSE 0 
    END as investment_experience_numeric
    
  FROM clients c
)

SELECT 
  fp.client_id,
  CURRENT_TIMESTAMP() as feature_timestamp,
  
  -- Financial features
  fp.age, fp.annual_income, fp.current_401k_balance, fp.years_to_retirement,
  fp.total_assets_under_management, fp.client_tenure_months,
  fp.income_to_age_ratio, fp.assets_to_income_ratio,
  ROUND(fp.retirement_readiness_score, 4) as retirement_readiness_score,
  ROUND(fp.wealth_growth_potential, 4) as wealth_growth_potential,
  fp.premium_client_indicator,
  fp.service_tier_numeric, fp.risk_tolerance_numeric, fp.investment_experience_numeric,
  
  -- Behavioral features
  COALESCE(cb.total_lifetime_events, 0) as total_lifetime_events,
  COALESCE(cb.engagement_span_days, 0) as engagement_span_days,
  COALESCE(cb.education_engagement, 0) as education_engagement,
  COALESCE(cb.advisor_meetings_total, 0) as advisor_meetings_total,
  
  -- Channel preference ratios
  ROUND(COALESCE(cb.web_preference_count, 0)::DECIMAL / GREATEST(cb.total_lifetime_events, 1), 4) as web_preference_ratio,
  ROUND(COALESCE(cb.email_preference_count, 0)::DECIMAL / GREATEST(cb.total_lifetime_events, 1), 4) as email_preference_ratio,
  ROUND(COALESCE(cb.phone_preference_count, 0)::DECIMAL / GREATEST(cb.total_lifetime_events, 1), 4) as phone_preference_ratio,
  ROUND(COALESCE(cb.inperson_preference_count, 0)::DECIMAL / GREATEST(cb.total_lifetime_events, 1), 4) as inperson_preference_ratio,
  
  -- Device adoption
  ROUND(COALESCE(cb.mobile_usage, 0)::DECIMAL / GREATEST((cb.mobile_usage + cb.desktop_usage), 1), 4) as mobile_adoption_score,
  
  -- Overall engagement frequency
  ROUND(COALESCE(cb.total_lifetime_events, 0)::DECIMAL / GREATEST(cb.engagement_span_days, 1), 4) as lifetime_engagement_frequency,
  
  -- Average value
  ROUND(COALESCE(cb.avg_touchpoint_value, 0), 4) as avg_touchpoint_value
  
FROM financial_profile fp
LEFT JOIN client_behaviors cb ON fp.client_id = cb.client_id
"""

# Execute feature creation
session.sql(financial_behavioral_sql).collect()

# Verify results
fb_count = session.sql("SELECT COUNT(*) as count FROM financial_behavioral_features").collect()[0]['COUNT']
print(f"✅ Created financial & behavioral features for {fb_count:,} clients")

# Show feature distributions
print("\n📈 Financial feature distributions:")
session.sql("""
    SELECT 
        ROUND(AVG(retirement_readiness_score), 4) as avg_retirement_readiness,
        ROUND(AVG(wealth_growth_potential), 4) as avg_wealth_potential,
        ROUND(AVG(mobile_adoption_score), 4) as avg_mobile_adoption,
        COUNT(CASE WHEN premium_client_indicator = 1 THEN 1 END) as premium_clients,
        ROUND(AVG(lifetime_engagement_frequency), 4) as avg_engagement_freq
    FROM financial_behavioral_features
""").show()


## Step 3: Create Target Variables & Lifecycle Features


In [ ]:
# Create target variables and lifecycle features
print("🎯 Creating target variables and lifecycle features...")

# First, check if engagement_features table exists
try:
    engagement_check = session.sql("""
        SELECT COUNT(*) as table_exists 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_NAME = 'ENGAGEMENT_FEATURES' 
        AND TABLE_SCHEMA = CURRENT_SCHEMA()
    """).collect()[0]['TABLE_EXISTS']
    
    if engagement_check == 0:
        print("❌ ERROR: engagement_features table not found!")
        print("   📋 Please run Step 1 (Create Engagement Features) first")
        print("   🔄 Run cell 3 to create the engagement_features table")
        raise Exception("Missing dependency: engagement_features table must be created first")
    else:
        print("✅ engagement_features table verified")
        
except Exception as e:
    if "Missing dependency" in str(e):
        raise e
    else:
        print(f"⚠️ Warning checking engagement_features: {e}")

targets_lifecycle_sql = """
CREATE OR REPLACE TABLE targets_lifecycle_features AS
WITH lifecycle_analysis AS (
  SELECT 
    c.client_id,
    c.client_tenure_months,
    c.age,
    c.service_tier,
    c.annual_income,
    c.total_assets_under_management,
    ef.days_since_last_activity,
    ef.engagement_score_30d,
    
    -- Lifecycle stage determination
    CASE 
      WHEN ef.days_since_last_activity IS NULL OR ef.days_since_last_activity > 180 THEN 'Dormant'
      WHEN ef.days_since_last_activity > 90 THEN 'At_Risk'
      WHEN c.client_tenure_months < 6 THEN 'New'
      WHEN c.client_tenure_months < 18 THEN 'Growing'
      ELSE 'Active'
    END as lifecycle_stage,
    
    -- Age segments
    CASE 
      WHEN c.age < 35 THEN 'Young'
      WHEN c.age < 50 THEN 'Mid-Career'
      WHEN c.age < 60 THEN 'Pre-Retirement'
      ELSE 'Near-Retirement'
    END as age_segment,
    
    -- Tenure segments
    CASE 
      WHEN c.client_tenure_months < 6 THEN 'New'
      WHEN c.client_tenure_months < 18 THEN 'Growing'
      WHEN c.client_tenure_months < 36 THEN 'Established'
      ELSE 'Mature'
    END as tenure_segment
    
  FROM clients c
  LEFT JOIN engagement_features ef ON c.client_id = ef.client_id
),

target_generation AS (
  SELECT 
    *,
    -- Conversion probability based on multiple factors
    LEAST(0.95, GREATEST(0.05,
      (CASE service_tier WHEN 'Elite' THEN 0.3 WHEN 'Premium' THEN 0.2 ELSE 0.1 END) +
      (CASE WHEN annual_income > 75000 THEN 0.2 ELSE 0.1 END) +
      (CASE WHEN total_assets_under_management > 50000 THEN 0.2 ELSE 0.1 END) +
      (COALESCE(engagement_score_30d, 0) * 0.3) +
      (UNIFORM(0, 0.1, RANDOM()))
    )) as conversion_probability,
    
    -- Churn probability (inverse relationship with conversion)
    LEAST(0.8, GREATEST(0.05,
      0.4 - 
      (CASE service_tier WHEN 'Elite' THEN 0.2 WHEN 'Premium' THEN 0.15 ELSE 0.05 END) -
      (COALESCE(engagement_score_30d, 0) * 0.2) +
      (CASE WHEN days_since_last_activity > 60 THEN 0.3 ELSE 0.0 END) +
      (UNIFORM(-0.1, 0.1, RANDOM()))
    )) as churn_probability
    
  FROM lifecycle_analysis
)

SELECT 
  client_id,
  CURRENT_TIMESTAMP() as feature_timestamp,
  
  -- Lifecycle features
  lifecycle_stage,
  age_segment,
  tenure_segment,
  days_since_last_activity,
  
  -- Target probabilities
  ROUND(conversion_probability, 4) as conversion_probability,
  ROUND(churn_probability, 4) as churn_probability,
  
  -- Binary targets (using probabilistic sampling)
  CASE WHEN UNIFORM(0, 1, RANDOM()) < conversion_probability THEN 1 ELSE 0 END as conversion_target,
  CASE WHEN UNIFORM(0, 1, RANDOM()) < churn_probability THEN 1 ELSE 0 END as churn_target,
  
  -- Next best action based on client profile
  CASE 
    WHEN service_tier = 'Basic' AND conversion_probability > 0.3 THEN 'Upgrade_Service_Tier'
    WHEN total_assets_under_management < 25000 AND conversion_probability > 0.25 THEN 'Schedule_Planning_Session'
    WHEN age_segment = 'Near-Retirement' AND conversion_probability > 0.2 THEN 'Retirement_Planning_Review'
    WHEN conversion_probability > 0.4 THEN 'Wealth_Advisory_Consultation'
    WHEN conversion_probability < 0.1 THEN 'Educational_Content'
    ELSE 'Relationship_Building'
  END as next_best_action,
  
  -- Business priority score
  ROUND(
    (conversion_probability * 0.4) + 
    ((1 - churn_probability) * 0.3) + 
    (CASE service_tier WHEN 'Elite' THEN 0.3 WHEN 'Premium' THEN 0.2 ELSE 0.1 END)
  , 4) as business_priority_score
  
FROM target_generation
"""

# Execute feature creation
session.sql(targets_lifecycle_sql).collect()

# Verify results
tl_count = session.sql("SELECT COUNT(*) as count FROM targets_lifecycle_features").collect()[0]['COUNT']
print(f"✅ Created target & lifecycle features for {tl_count:,} clients")

# Show target distributions
print("\n🎲 Target variable distributions:")
session.sql("""
    SELECT 
        lifecycle_stage,
        COUNT(*) as client_count,
        ROUND(AVG(conversion_probability), 4) as avg_conversion_prob,
        ROUND(AVG(churn_probability), 4) as avg_churn_prob,
        SUM(conversion_target) as conversion_targets,
        SUM(churn_target) as churn_targets
    FROM targets_lifecycle_features
    GROUP BY lifecycle_stage
    ORDER BY client_count DESC
""").show()

print("\n📋 Next best action distribution:")
session.sql("""
    SELECT 
        next_best_action,
        COUNT(*) as client_count,
        ROUND(AVG(business_priority_score), 4) as avg_priority_score
    FROM targets_lifecycle_features
    GROUP BY next_best_action
    ORDER BY client_count DESC
""").show()


## Step 4: Create Unified Feature Store


In [ ]:
# Create unified feature store combining all feature sets
print("🏪 Creating unified feature store...")

# Check for all required dependencies
required_tables = ['engagement_features', 'financial_behavioral_features', 'targets_lifecycle_features']
missing_tables = []

for table in required_tables:
    try:
        table_check = session.sql(f"""
            SELECT COUNT(*) as table_exists 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = '{table.upper()}' 
            AND TABLE_SCHEMA = CURRENT_SCHEMA()
        """).collect()[0]['TABLE_EXISTS']
        
        if table_check == 0:
            missing_tables.append(table)
        else:
            print(f"✅ {table} table verified")
            
    except Exception as e:
        print(f"⚠️ Warning checking {table}: {e}")
        missing_tables.append(table)

if missing_tables:
    print("❌ ERROR: Missing required feature tables!")
    for table in missing_tables:
        print(f"   📋 Missing: {table}")
    print("   🔄 Please run all previous feature engineering cells first")
    raise Exception(f"Missing dependencies: {', '.join(missing_tables)}")

print("✅ All feature tables verified - proceeding with unified feature store creation")

unified_feature_store_sql = """
CREATE OR REPLACE TABLE feature_store AS
SELECT 
  ef.client_id,
  ef.feature_timestamp,
  
  -- Engagement features
  ef.total_events_7d, ef.web_visits_7d, ef.email_opens_7d, ef.email_clicks_7d,
  ef.total_events_30d, ef.web_visits_30d, ef.email_opens_30d, ef.email_clicks_30d, ef.personal_interactions_30d,
  ef.total_events_90d, ef.web_visits_90d,
  ef.avg_session_duration_30d, ef.active_days_30d,
  ef.total_touchpoint_value_30d, ef.avg_touchpoint_value_30d, ef.conversions_30d,
  ef.days_since_last_activity, ef.engagement_frequency_30d, ef.email_click_rate_30d,
  ef.engagement_trend_30d, ef.engagement_score_30d,
  
  -- Financial & behavioral features
  fbf.age, fbf.annual_income, fbf.current_401k_balance, fbf.years_to_retirement,
  fbf.total_assets_under_management, fbf.client_tenure_months,
  fbf.income_to_age_ratio, fbf.assets_to_income_ratio,
  fbf.retirement_readiness_score, fbf.wealth_growth_potential, fbf.premium_client_indicator,
  fbf.service_tier_numeric, fbf.risk_tolerance_numeric, fbf.investment_experience_numeric,
  fbf.total_lifetime_events, fbf.engagement_span_days, fbf.education_engagement, fbf.advisor_meetings_total,
  fbf.web_preference_ratio, fbf.email_preference_ratio, fbf.phone_preference_ratio, fbf.inperson_preference_ratio,
  fbf.mobile_adoption_score, fbf.lifetime_engagement_frequency, fbf.avg_touchpoint_value,
  
  -- Lifecycle & target features
  tlf.lifecycle_stage, tlf.age_segment, tlf.tenure_segment,
  tlf.conversion_probability, tlf.churn_probability,
  tlf.conversion_target, tlf.churn_target, tlf.next_best_action,
  tlf.business_priority_score
  
FROM engagement_features ef
LEFT JOIN financial_behavioral_features fbf ON ef.client_id = fbf.client_id
LEFT JOIN targets_lifecycle_features tlf ON ef.client_id = tlf.client_id
WHERE ef.client_id IS NOT NULL
"""

# Execute unified feature store creation
session.sql(unified_feature_store_sql).collect()

# Verify and analyze feature store
fs_count = session.sql("SELECT COUNT(*) as count FROM feature_store").collect()[0]['COUNT']
feature_count = session.sql("SELECT COUNT(*) as feature_count FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'FEATURE_STORE'").collect()[0]['FEATURE_COUNT']

print(f"✅ Created unified feature store:")
print(f"   📊 Records: {fs_count:,} clients")
print(f"   🔧 Features: {feature_count} total features")

# Feature completeness analysis
print("\n🔍 Feature completeness analysis:")
session.sql("""
    SELECT 
        COUNT(*) as total_records,
        COUNT(CASE WHEN engagement_score_30d IS NOT NULL THEN 1 END) as with_engagement_score,
        COUNT(CASE WHEN retirement_readiness_score IS NOT NULL THEN 1 END) as with_retirement_score,
        COUNT(CASE WHEN conversion_target IS NOT NULL THEN 1 END) as with_conversion_target,
        COUNT(CASE WHEN churn_target IS NOT NULL THEN 1 END) as with_churn_target,
        ROUND(
            COUNT(CASE WHEN engagement_score_30d IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 2
        ) as completeness_percentage
    FROM feature_store
""").show()

# Feature statistics
print("\n📈 Key feature statistics:")
session.sql("""
    SELECT 
        ROUND(AVG(engagement_score_30d), 4) as avg_engagement_score,
        ROUND(AVG(retirement_readiness_score), 4) as avg_retirement_readiness,
        ROUND(AVG(conversion_probability), 4) as avg_conversion_prob,
        ROUND(AVG(churn_probability), 4) as avg_churn_prob,
        SUM(conversion_target) as total_conversion_targets,
        SUM(churn_target) as total_churn_targets
    FROM feature_store
""").show()


## Step 5: Initialize Snowflake Feature Store


In [ ]:
# Cell 10: Verify Features are Ready
print("=== Feature Engineering Complete ===\n")

# Verify feature table
try:
    feature_count = session.table("FEATURE_STORE").count()
    feature_cols = len(session.table("FEATURE_STORE").columns)
    
    print(f"✅ FEATURE_STORE table:")
    print(f"   - Records: {feature_count:,}")
    print(f"   - Columns: {feature_cols}")
    print(f"   - Location: {session.get_current_database()}.{session.get_current_schema()}.FEATURE_STORE")
    
    # Show sample features
    print("\n📈 Sample High-Value Clients:")
    session.table("FEATURE_STORE").filter(
        "WEALTH_GROWTH_POTENTIAL > 0.7"
    ).select(
        "CLIENT_ID",
        "ENGAGEMENT_SCORE_30D",
        "RETIREMENT_READINESS_SCORE",
        "ANNUAL_INCOME",
        "LIFECYCLE_STAGE"
    ).limit(5).show()
    
    print("\n✅ FEATURES READY FOR MODEL TRAINING!")
    print("\n🚀 Next Steps:")
    print("1. Optional: Run Cell 11 to register in Feature Store UI")
    print("2. Proceed to 03_Model_Training_Registry_Snowflake.ipynb")
    print("3. Use: features_df = session.table('FEATURE_STORE')")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you've run cells 1-9 in order!")

## Optional: Register in Snowflake Feature Store UI

The cell below registers your features in Snowflake's Feature Store for UI visibility. This is optional - your features are already ready for ML training in the FEATURE_STORE table.

In [ ]:
# Cell 11: Register in Snowflake Feature Store (Optional)
print("Registering features in Snowflake Feature Store...")

from snowflake.ml.feature_store import FeatureStore, Entity, FeatureView, CreationMode
import pandas as pd

# Create a new database to avoid any corruption issues
new_db = f"FINANCIAL_ML_DEMO_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
print(f"Creating new database: {new_db}")

session.sql(f"CREATE DATABASE IF NOT EXISTS {new_db}").collect()
session.sql(f"USE DATABASE {new_db}").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS ML_PIPELINE").collect()
session.sql("USE SCHEMA ML_PIPELINE").collect()

# Copy your feature table to the new database
print("Copying features to new database...")
original_db = "FINANCIAL_ML_DB"  # Update if different
session.sql(f"""
    CREATE TABLE FEATURE_STORE AS 
    SELECT * FROM {original_db}.ML_PIPELINE.FEATURE_STORE
""").collect()

# Now register Feature Store in the clean database
fs = FeatureStore(
    session=session,
    database=new_db,
    name="FEATURE_STORE",
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# Register entity and feature view
client_entity = Entity(name="client", join_keys=["CLIENT_ID"])
fs.register_entity(client_entity)

client_fv = FeatureView(
    name="client_features_v1",
    entities=[client_entity],
    feature_df=session.table("FEATURE_STORE")
)

fs.register_feature_view(client_fv, version="1.0")

print(f"\n✅ SUCCESS! Feature Store registered in: {new_db}")
print("Check Snowsight → AI & ML → Feature Store")
print(f"\n📝 IMPORTANT: Update your Model Training notebook to use:")
print(f"   Database: {new_db}")
print(f"   Table: {new_db}.ML_PIPELINE.FEATURE_STORE")
print(f"\nSave this database name: {new_db}")